# Dataset cleaning and normalization

After a while of using and analyzing the imported dataset I noticed many issues with it.

I am creating this notebook to improve the dataset's quality and to help anyone that wishes to use it.

## Data Cleaning Process

Our Data cleaning Process will follow these steps:

1. First for each column we will do some statistics to understand the data's structure, detect issues, and check any possible improvements.
2. After that we will proceed with cleaning and normalizing the data depending on the case.
 

### Contact Me

Please if you notice anything that is lacking or spot an issue with this notebook or the dataset, contact me through one of these accounts: 

<style>
  .contact-container {
    display: flex;
    justify-content: center;
    margin: 20px 0;
  }

  .contact-table {
    border-collapse: collapse;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    width: 100%;
    max-width: 500px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
  }

  .contact-table td {
    padding: 15px;
    vertical-align: middle;
  }

  .icon-cell {
    background-color: #f5f5f5;
    width: 60px;
    text-align: center;
    border-right: 1px solid #e0e0e0;
  }

  .icon-cell img {
    width: 30px;
    height: 30px;
  }

  .info-cell a {
    text-decoration: none;
    color: #2c3e50;
    font-size: 16px;
    display: inline-block;
  }

  .info-cell a:hover {
    color: #2980b9;
    text-decoration: underline;
  }

  @media (max-width: 600px) {
    .contact-table, .contact-table td {
      display: block;
      width: 100%;
    }

    .icon-cell {
      border-right: none;
      border-bottom: 1px solid #e0e0e0;
    }
  }
</style>

<div class="contact-container">
  <table class="contact-table">
    <tr>
      <td class="icon-cell">
        <img src="https://img.icons8.com/color/48/000000/linkedin.png" alt="LinkedIn"/>
      </td>
      <td class="info-cell">
        <a href="https://www.linkedin.com/in/addalaraed/" target="_blank">Raed Addala</a>
      </td>
    </tr>
    <tr>
      <td class="icon-cell">
        <img src="https://img.icons8.com/color/48/000000/twitter.png" alt="Twitter"/>
      </td>
      <td class="info-cell">
        <a href="https://x.com/AddalaRaed" target="_blank">@AddalaRaed</a>
      </td>
    </tr>
    <tr>
      <td class="icon-cell">
        <img src="https://img.icons8.com/color/48/000000/email.png" alt="Email"/>
      </td>
      <td class="info-cell">
        <a href="mailto:addala.raed@gmail.com">addala.raed@gmail.com</a>
      </td>
    </tr>
    <tr>
      <td class="icon-cell">
        <img src="https://img.icons8.com/material-outlined/48/000000/github.png" alt="GitHub"/>
      </td>
      <td class="info-cell">
        <a href="https://github.com/RaedAddala" target="_blank">RaedAddala</a>
      </td>
    </tr>
  </table>
</div>


## Dataset Stats

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("/kaggle/input/top-500-600-movies-of-each-year-from-1960-to-2024/final_dataset.csv")

In [3]:
def describe(df):
    print_size = 50
    # Display data types
    print("="*print_size)
    print("\nData Types:")
    print("-"*print_size)
    print(df.dtypes)
    # Get summary statistics for all columns
    print("="*print_size)
    print("\nSummary Statistics:")
    print("-"*print_size)
    print(df.describe(include='all'))
    print("="*print_size)
    # Check for missing values
    print("\nMissing Values:")
    print("-"*print_size)
    print(df.isnull().sum())
    print("="*print_size)

In [4]:
describe(df)


Data Types:
--------------------------------------------------
id                        object
title                     object
year                       int64
duration                  object
MPA                       object
rating                   float64
votes                     object
meta_score               float64
description               object
Movie_Link                object
writers                   object
directors                 object
stars                     object
budget                    object
opening_weekend_gross     object
gross_worldwide           object
gross_us_canada           object
release_date             float64
countries_origin          object
filming_locations         object
production_companies      object
awards_content            object
genres                    object
languages                 object
dtype: object

Summary Statistics:
--------------------------------------------------
               id                 title          year dura

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


### Debugging Functions

Please note that these functions defined here were used in the developement process for testing and checking if the cleaning is working as intended or not. They were verbose in terms of logs so I removed their calls in the final version but kept the functions there for testing purposes in the future.


In [5]:
def analyze_column(df, column_name):
    # Handle nulls
    nulls = df[df[column_name].isnull()]
    print(f"\n🔍 Null values in '{column_name}': {len(nulls)}")
    # Handle duplicates
    duplicates = df[column_name][df[column_name].duplicated(keep=False)]
    print(f"\n🔁 Duplicate values in '{column_name}':")
    if not duplicates.empty:
        print(duplicates.value_counts())
    else:
        print("No duplicates found.")

In [6]:
def is_array_like(col):
    non_null = col.dropna()
    if non_null.empty:
        return False
    return non_null.apply(lambda x: isinstance(x, list) or isinstance(x, np.ndarray)).all()

def print_unique_values(df):
    for col in df.columns:
        if is_array_like(df[col]):
            all_values = [item for sublist in df[col].dropna() for item in sublist if pd.notna(item)]
            series = pd.Series(all_values)
            if pd.api.types.is_numeric_dtype(series):
                print(f"Statistics for numerical array-like column '{col}':")
                print(series.describe())
            else:
                unique_vals = series.unique()
                print(f"Unique values for textual array-like column '{col}': {unique_vals}")
        else:
            if pd.api.types.is_numeric_dtype(df[col]):
                print(f"Statistics for numerical column '{col}':")
                print(df[col].describe())
            elif df[col].dtype == 'object':
                unique_vals = df[col].unique()
                print(f"Unique values for textual column '{col}': {unique_vals}")
            else:
                print(f"Column '{col}' has dtype {df[col].dtype}, not handled.")

## Data Cleaning
I will proceed with each column on its own now.

In the developement of this notebook and during the cleaning process, I used the functions above to study each column before proceeding.

### ID

In [7]:
df["id"] = df["id"].astype("string")
df['id'] = df['id'].apply(lambda x: x.strip().lower())

### TITLE

In [8]:
df["title"] = df["title"].astype("string")
df['title'] = df['title'].apply(lambda x: x.strip().lower())

### Year

In [9]:
df['year'] = pd.to_numeric(df['year'], downcast='integer')

### Duration and MPA

In [10]:
def is_h_m_format(val):
    """Check if value is in a valid duration format"""
    if pd.isna(val):
        return True  # Consider NaNs as "valid" for now
    
    val = str(val).strip().lower()
    patterns = [
        r'(?:(\d+)\s*h)?\s*(\d+)\s*m',         # "1h 30m", "30m"
        r'(\d+)\s*min',                        # "90 min"
        r'(\d+)\s*minutes?',                   # "45 minutes"
        r'(\d+)\s*hours?',                     # "2 hours"
        r'^(\d+):(\d+)$',                      # "1:15"
        r'^(\d+)$',                            # just a number, like "90"
        r'^(\d+)\s*h$'                         # "2h"
    ]
    
    for pattern in patterns:
        if re.fullmatch(pattern, val):
            return True
    return False

def convert_to_minutes(val):
    """Convert various duration formats to minutes (integer)"""
    if pd.isna(val):
        return -1
    
    # If the value is already an integer or float, just return it
    if isinstance(val, (int, float)) and not pd.isna(val):
        return int(val)
    
    val = str(val).strip().lower()
    
    # Pattern matching for different time formats
    # Format: "1h 30m" or "30m"
    match = re.fullmatch(r'(?:(\d+)\s*h)?\s*(\d+)\s*m', val)
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2))
        return hours * 60 + minutes
    
    # Format: "90 min"
    match = re.fullmatch(r'(\d+)\s*min', val)
    if match:
        return int(match.group(1))
    
    # Format: "45 minutes"
    match = re.fullmatch(r'(\d+)\s*minutes?', val)
    if match:
        return int(match.group(1))
    
    # Format: "2 hours"
    match = re.fullmatch(r'(\d+)\s*hours?', val)
    if match:
        return int(match.group(1)) * 60
    
    # Format: "1:15"
    match = re.fullmatch(r'^(\d+):(\d+)$', val)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return hours * 60 + minutes
    
    # Just a number, like "90"
    match = re.fullmatch(r'^(\d+)$', val)
    if match:
        return int(match.group(1))
    
    # Format: "2h"
    match = re.fullmatch(r'^(\d+)\s*h$', val)
    if match:
        return int(match.group(1)) * 60
    
    return -1

In [11]:
# Define simplified MPA standardization mapping (grouped by minimum age)
mpa_standardization = {
    # Group G: General Audiences (Age 0+)
    'g': 'general',
    'tv-g': 'general',
    'tv-y': 'general',
    
    # Group PG: Parental Guidance (Age 7-10+)
    'pg': 'parental_guidance',
    'tv-pg': 'parental_guidance',
    'approved': 'parental_guidance',
    'passed': 'parental_guidance',
    'm': 'parental_guidance',
    'm/pg': 'parental_guidance',
    'gp': 'parental_guidance',
    'tv-y7': 'parental_guidance',
    'tv-y7-fv': 'parental_guidance',
    
    # Group Teen: Teen Content (Age 13-14+)
    'pg-13': 'teen',
    'tv-14': 'teen',
    'tv-13': 'teen',
    '13+': 'teen',
    
    # Group Mature: Mature Content (Age 16-17+)
    'r': 'mature',
    'tv-ma': 'mature',
    'ma-17': 'mature',
    '16+': 'mature',
    
    # Group Adult: Adults Only (Age 18+)
    'nc-17': 'adults_only',
    'x': 'adults_only',
    '18+': 'adults_only',
    
    # Unrated group
    'not rated': 'unrated',
    'unrated': 'unrated',
    'none': 'unrated',
    'e': 'unrated'
}

# Define minimum age mapping based on simplified categories
min_age_map = {
    'general': 0,          # G, TV-G, TV-Y
    'parental_guidance': 7, # PG, TV-PG, Approved, TV-Y7
    'teen': 13,            # PG-13, TV-14
    'mature': 17,          # R, TV-MA
    'adults_only': 18,     # NC-17, X
    'unrated': 'unrated'
}

In [12]:
# Step 1: Fix swapped values between duration and MPA
print("Step 1: Identifying and fixing swapped values between duration and MPA")
# First identify any known rating values that are in the duration column
known_ratings = set(mpa_standardization.keys())
df['duration_normalized'] = df['duration'].apply(lambda x: str(x).strip().lower() if not pd.isna(x) else '')

# Show invalid durations
invalid_durations = df[~df['duration'].apply(is_h_m_format)]
print("\n❌ Durations with unrecognized format:")
print(invalid_durations['duration'].unique())

print("\n🔀 Mix with MPA for invalid durations (including nulls):")
print(
    invalid_durations
    .groupby(['duration', 'MPA'], dropna=False)
    .size()
    .sort_index()
)

Step 1: Identifying and fixing swapped values between duration and MPA

❌ Durations with unrecognized format:
['R' 'Passed' 'PG-13' 'TV-14' 'PG' '18+' 'TV-PG' 'Approved' 'E']

🔀 Mix with MPA for invalid durations (including nulls):
duration  MPA
18+       NaN     1
Approved  NaN     4
E         NaN     1
PG        NaN     3
PG-13     NaN     3
Passed    NaN    54
R         NaN    19
TV-14     NaN     2
TV-PG     NaN     1
dtype: int64


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


**Observations:**
- misplaced values in duration that must be put in MPA.
- there is a value for unrated movies and so we can substitute nan with unrated **(Note this decision came from the fact that if IMDB didn't contain the rating then getting it and verifying it from elsewhere wouldn't add more information. If you can do better then do so. In my case I unified many values in MPA.)**.

In [13]:
print("\nBefore Swap")
print(f"Null values in MPA: {df['MPA'].isna().sum()}")
print(df['MPA'].value_counts().sort_values(ascending=False).head(20))

# Identify rows where duration contains a rating value and MPA is null
mask = df['duration_normalized'].isin(known_ratings) & df['MPA'].isna()
# Swap the values for these rows
df.loc[mask, ['duration', 'MPA']] = df.loc[mask, ['MPA', 'duration']].values

# Clean up temporary column
df.drop(columns=['duration_normalized'], inplace=True)

print("\nAfter Swap")
print(f"Null values in MPA: {df['MPA'].isna().sum()}")
print(df['MPA'].value_counts().sort_values(ascending=False).head(20))


Before Swap
Null values in MPA: 22022
MPA
R            11339
Approved     10000
Not Rated     6450
PG-13         4014
PG            3770
Passed        2128
Unrated       1149
G              901
TV-MA          361
TV-14          264
TV-PG          228
X              173
GP             137
TV-G            96
NC-17           68
M               57
M/PG            45
TV-Y7           13
18+             10
16+              9
Name: count, dtype: int64

After Swap
Null values in MPA: 21934
MPA
R            11358
Approved     10004
Not Rated     6450
PG-13         4017
PG            3773
Passed        2182
Unrated       1149
G              901
TV-MA          361
TV-14          266
TV-PG          229
X              173
GP             137
TV-G            96
NC-17           68
M               57
M/PG            45
TV-Y7           13
18+             11
16+              9
Name: count, dtype: int64


In [14]:
# Step 2: Process duration values
print("\nStep 2: Processing duration values")
# Convert duration to minutes
df['duration_minutes'] = df['duration'].apply(convert_to_minutes)
# Set duration_converted flag (True if conversion succeeded, False if resulted in -1)
df['duration_converted'] = df['duration_minutes'] != -1
# Replace original duration with minutes
df['duration'] = df['duration_minutes']
# Clean up temporary column
df.drop(columns=['duration_minutes'], inplace=True)

print(f"Successfully converted durations: {df['duration_converted'].sum()} rows")
print(f"Failed conversions: {(~df['duration_converted']).sum()} rows")


Step 2: Processing duration values
Successfully converted durations: 61086 rows
Failed conversions: 2163 rows


In [15]:
# Step 3: Process MPA values
print("\nStep 3: Processing MPA values")
# Keep original values for reference
df['MPA_original'] = df['MPA'].copy()

# Normalize MPA values (fill with unrated, lowercase, strip, standardize)
df['MPA'] = df['MPA'].fillna('unrated')
df['MPA'] = df['MPA'].apply(lambda x: str(x).strip().lower() if pd.notna(x) else 'unrated')
df['MPA'] = df['MPA'].apply(lambda x: mpa_standardization.get(x, 'unrated'))

# Add minimum age column based on standardized MPA
df['MPA_min_age'] = df['MPA'].apply(lambda x: min_age_map.get(x, 'unrated'))

print("\nMPA standardization summary:")
print(df['MPA'].value_counts().sort_values(ascending=False))
print("\nMinimum age distribution:")
print(df['MPA_min_age'].value_counts().sort_values(ascending=False))


Step 3: Processing MPA values

MPA standardization summary:
MPA
unrated              29534
parental_guidance    16445
mature               11729
teen                  4289
general               1000
adults_only            252
Name: count, dtype: int64

Minimum age distribution:
MPA_min_age
unrated    29534
7          16445
17         11729
13          4289
0           1000
18           252
Name: count, dtype: int64


In [16]:
# Step 4: Optimize data types
print("\nStep 4: Optimizing data types")
# Convert duration to optimal integer type
df['duration'] = pd.to_numeric(df['duration'], downcast='integer')
# Ensure duration_converted is boolean type
df['duration_converted'] = df['duration_converted'].astype('bool')
# Make MPA columns categorical for efficiency
df['MPA'] = df['MPA'].astype('category')
df['MPA_original'] = df['MPA_original'].astype('category')

# Handle MPA_min_age as a mixed-type column (numeric and 'unrated')
numeric_mask = df['MPA_min_age'] != 'unrated'
if numeric_mask.any():
    df.loc[numeric_mask, 'MPA_min_age'] = pd.to_numeric(df.loc[numeric_mask, 'MPA_min_age'], downcast='integer')
df['MPA_min_age'] = df['MPA_min_age'].astype('category')


Step 4: Optimizing data types


In [17]:
# Final summary
print("\nFinal Data Summary:")
print(f"Total rows: {len(df)}")
print(f"MPA null values: {df['MPA'].isna().sum()}")
print(f"Duration range: {df['duration'].min()} to {df['duration'].max()} minutes")
print(f"Duration conversion success rate: {df['duration_converted'].mean()*100:.2f}%")
print(f"Unique MPA categories: {df['MPA'].nunique()}")
print(f"Unique MPA_min_age values: {df['MPA_min_age'].nunique()}")


Final Data Summary:
Total rows: 63249
MPA null values: 0
Duration range: -1 to 5220 minutes
Duration conversion success rate: 96.58%
Unique MPA categories: 6
Unique MPA_min_age values: 6


***Done:*** Cleaning, Normalizing, optimizing these columns and adding more columns to the dataset.

### Rating and Votes

In [18]:
analyze_column(df, "rating")
analyze_column(df, "votes")


🔍 Null values in 'rating': 4068

🔁 Duplicate values in 'rating':
rating
6.3    2525
6.4    2498
6.2    2456
6.5    2417
6.6    2407
       ... 
1.3       5
9.6       4
9.5       3
1.4       3
1.2       2
Name: count, Length: 86, dtype: int64

🔍 Null values in 'votes': 4068

🔁 Duplicate values in 'votes':
votes
1.1K    835
1.2K    776
1.4K    633
1.3K    633
1.5K    574
       ... 
478K      2
628K      2
369K      2
595K      2
348K      2
Name: count, Length: 1559, dtype: int64


***Observations:***
- votes and rating are null values together ( as it should be ).
- votes is set as a string, it would be better to put it as an integer so conversion is needed.

In [19]:
df['rating'] = df['rating'].fillna(-1)
df['rating'] = pd.to_numeric(df['rating'], downcast='float')
# Define all allowed categories
rating_dtype = df['rating'].dtype
valid_ratings = np.round(np.arange(0.0, 10.1, 0.1), 1).astype(rating_dtype).tolist()
valid_ratings.insert(0, -1.0)

df['rating_as_category'] = pd.Categorical(df['rating'], categories=valid_ratings, ordered=True)

def parse_votes(val):
    try:
        if isinstance(val, str):
            val = val.strip().lower()
            if val.endswith('k'):
                return int(float(val[:-1]) * 1_000)
            elif val.endswith('m'):
                return int(float(val[:-1]) * 1_000_000)
            else:
                return int(val)
        elif pd.isna(val):
            return -1
        else:
            return int(val)
    except:
        return -1

df['votes'] = df['votes'].apply(parse_votes)
df['votes'] = pd.to_numeric(df['votes'], downcast='integer')

df['rating_was_missing'] = ((df['rating'] == -1) | (df['votes'] == -1))
df['rating_was_missing'] = df['rating_was_missing'].astype('bool')

### Meta_Score
The Meta Score typically refers to an aggregated score or rating that represents the critical consensus for a piece of media.

IMDb's MetaScore aggregates reviews from professional critics and provides a numerical score on a scale (e.g., 0–100).

In [20]:
analyze_column(df,"meta_score")
df['meta_score'] = df['meta_score'].fillna(-1)
df['meta_score'] = pd.to_numeric(df['meta_score'], downcast='integer')
df['meta_score_was_missing'] = (df['meta_score'] == -1)
df['meta_score_was_missing'] = df['meta_score_was_missing'].astype('bool')
analyze_column(df,"meta_score")


🔍 Null values in 'meta_score': 47716

🔁 Duplicate values in 'meta_score':
meta_score
66.0    371
65.0    367
64.0    366
68.0    366
72.0    345
       ... 
1.0       8
7.0       7
8.0       4
5.0       3
4.0       3
Name: count, Length: 97, dtype: int64

🔍 Null values in 'meta_score': 0

🔁 Duplicate values in 'meta_score':
meta_score
-1     47716
 66      371
 65      367
 64      366
 68      366
       ...  
 1         8
 7         7
 8         4
 5         3
 4         3
Name: count, Length: 98, dtype: int64
